## 第二步
1. 移动宝宝目录下的ios文件至新地址xx
    > \\192.168.31.99\photo\宝宝\hanminmin\YYYYMMDD\ios
2. 遍历xx目录下的文件 区分日期目录 标记为ios
3. 遍历相机胶卷目录下的全部文件 区分日期目录 标记为win
    > \\192.168.31.99\photo\hanminmin\photo\相机胶卷\YYYYMMDD
4. merger相同目录下的文件，字段：file_name file_path source_flag dir_name
5. 修改并移动相同目录下(dir_name)，相同文件名(file_name)的文件到xx目录下（日期根目录-》ios、win）
6. 人工判断ios标记目录下的重复文件，进行去重操作，删除win标记文件（为压缩文件）
7. 人工判断剩余win标记文件夹下的文件，删除无用文件
8. 移动win标记文件至xx标记目录下
    > \\192.168.31.99\photo\宝宝\hanminmin\YYYYMMDD\win

In [1]:
"""
第一步
目标：
    1、获取ios文件 数据矩阵
    2、获取win文件 数据矩阵
    3、通过 日期文件夹名称、文件名称 merger ios、win数据矩阵
"""
import os, pandas as pd

def df_for_datedir(dir_path, date_dir_name):
    print("开始执行文件夹：%s" % dir_path)
    file_name_list = []
    file_path_list = []
    spli_file_list = []
    date_dir_name_list = []
    file_list = os.listdir(dir_path)
    file_lag = os.path.basename(dir_path)
    for file in file_list:
        file_path = os.path.join(dir_path, file)
        if os.path.isfile(file_path):
            spli_file = os.path.splitext(file)
            spli_file_list.append(','.join(spli_file))
            file_name_list.append(file)
            date_dir_name_list.append(date_dir_name)
            file_path_list.append(file_path)
        else:
            print("警告：=================日期文件夹下出现了二级目录：%s" % file_path)

    df = pd.DataFrame({"file_path":file_path_list, "spli_file":spli_file_list, "file_name": file_name_list, "dir_name": date_dir_name_list})
    print("结束执行文件夹：%s" % dir_path)
    return df

# df = df_for_datedir(r"\\192.168.31.99\photo\宝宝\hanminmin\202003\ios", "202003")


In [2]:
import threading, queue

global df
df = pd.DataFrame()
def add_df(ios_dir_path, date_dir_name):
    df_datedir = df_for_datedir(ios_dir_path, date_dir_name)
    print("df_datedir======================="+str(len(df_datedir)))
    global df
    print("df======================="+str(len(df)))
    if df.empty:
        df = df_datedir
    else:
        df = df.append(df_datedir, ignore_index = True)
    print("df======================="+str(len(df)))

# 从队列中取出值，并调用实际操作
def customer(thread_name, my_queue):
    print("=========启动一个thread-%d" % thread_name)
    while not my_queue.empty():
        list = my_queue.get()
        print("thread-%d开始执行add_df()方法,入参dir_path:%s,date_dir_name:%s=====" % (thread_name, list[0], list[1],))
        add_df(list[0], list[1])
    print("=========结束线程thread-%d" % thread_name)

def thread_queue(work_path, thread_count, file_flag):
    print("=========================开始")
    # 创建一个队列对象，把数组值放进去
    dir_list = os.listdir(work_path)
    my_queue = queue.Queue(maxsize=len(dir_list))
    for dir in dir_list:
        # 文件在YYYYMMDD目录下的ios目录下
        ios_dir_path = os.path.join(work_path, dir, file_flag)
        list = [ios_dir_path, dir]
        my_queue.put(list)

    # 起10个线程，线程target去执行从队列中取值并进行操作的动作
    threads = []
    print("=========================添加线程池")
    for t in range(thread_count):
        thread = threading.Thread(target=customer, args=(t, my_queue,))
        threads.append(thread)
        thread.start()

    print("=========================启动线程池")
    for t in threads:
        t.join()
    print("success")
    return df


In [ ]:
# 执行前先初始化df，再执行ios的数据
df = pd.DataFrame()
ios_dir_path = r"\\192.168.31.99\photo\宝宝\hanminmin"
df_baby = thread_queue(ios_dir_path, 10, 'ios')

In [ ]:
# df已存储baby的历史数据，先初始化df再执行win的数据
df = pd.DataFrame()
win_dir_path = r"\\192.168.31.99\photo\hanminmin\photo\相机胶卷"
df_win = thread_queue(win_dir_path, 10, 'win')

将数据集合利用pandas存入sqlite中

In [4]:
# 同一目录下
# from db_demo import connect_db,close_db

# 不同目录下
import sys
sys.path.append("../../")
from db.db_demo import connect_db


db_file = r'E:/SQLite_DB/nas_photo.db'
conn = connect_db(db_file)
df_baby.to_sql(name='df_baby',con=conn,if_exists='replace',index=False)
sql = "select * from df_baby"
#完成数据库的查询读取到数据框dataframe中
df_baby_db = pd.read_sql(sql,conn)
conn.close

<function Connection.close>

In [6]:
db_file = r'E:/SQLite_DB/nas_photo.db'
conn = connect_db(db_file)
df_win.to_sql(name='df_win',con=conn,if_exists='replace',index=False)
sql = "select * from df_win"
#完成数据库的查询读取到数据框dataframe中
df_win_db = pd.read_sql(sql,conn)
conn.close

<function Connection.close>

list与字符串互转

In [4]:
path = r'\\192.168.31.99\photo\宝宝\hanminmin\202003\ios\IMG_0004.JPG'
spilt = os.path.splitext(path)
str_spilt = ','.join(spilt)
print(str_spilt)
list = str_spilt.split(',')
for name in list:
    print(name)

\\192.168.31.99\photo\宝宝\hanminmin\202003\ios\IMG_0004,.JPG
\\192.168.31.99\photo\宝宝\hanminmin\202003\ios\IMG_0004
.JPG
